In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pennylane as qml
from pennylane import numpy as np
import os
from matplotlib import pyplot as plt

comp_device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
project_dir = os.getcwd()
data_dir = os.path.join(project_dir, 'data')

In [3]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
"""
PLOTTING FUNCTIONS
"""

def plot_loss_metric(loss, val_loss, metric, val_metric, metric_name='accuracy'):

    epochs_range = range(len(loss))

    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    if metric_name == 'accuracy':
        plt.plot(epochs_range, metric, label='Training Accuracy')
        plt.plot(epochs_range, val_metric, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
    elif metric_name == 'recall':
        plt.plot(epochs_range, metric, label='Training Recall')
        plt.plot(epochs_range, val_metric, label='Validation Recall')
        plt.title('Training and Validation Recall')
    elif metric_name == 'precision':
        plt.plot(epochs_range, metric, label='Training Precision')
        plt.plot(epochs_range, val_metric, label='Validation Precision')
        plt.title('Training and Validation Precision')
    plt.legend(loc='lower right')
    

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [5]:
batch_size = 1
epochs = 50
learning_rate = 1e-3
drop_rate = 0.5
n_qbits = 9

q_device = qml.device("default.qubit", wires=n_qbits)

In [6]:
training_data = datasets.MNIST(
    root=data_dir,
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root=data_dir,
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

train_dl = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size)

In [7]:
"""
quantum conv
"""

class QuantumConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        weights_tensor = torch.Tensor(self.out_channels, self.in_channels*(self.kernel_size)**2)
        self.weights = nn.Parameter(weights_tensor)
        nn.init.xavier_uniform_(self.weights)


    def compute_windows(self, x):
        return nn.functional.unfold(x, self.kernel_size, padding=1)


    @qml.qnode(q_device, interface="torch")
    def quantum_circuit(x, weights):

        weights = torch.unsqueeze(weights, 0)
        qml.AngleEmbedding(features=x, wires=range(n_qbits), rotation='X')
        qml.BasicEntanglerLayers(weights=weights, wires=range(n_qbits))

        return [qml.expval(qml.PauliZ(j)) for j in range(n_qbits)]

    
    def quantum_conv(self, x, weights):
        
        measurements = self.quantum_circuit(x, weights)
        return torch.mean(measurements)


    def forward(self, x):
        
        padded_input = nn.functional.pad(x, (1,1,1,1), value=0.)
        output_tensor = torch.zeros(
            [x.shape[0], self.out_channels, 28*28],
            dtype = torch.float32
        )
        
        windows = self.compute_windows(padded_input)

        for ibatch in range(x.shape[0]):
            for chan_out in range(self.out_channels):

                for window in range(28*28):
                    qconv = self.quantum_conv(windows[ibatch,:,window], self.weights[chan_out])
                    output_tensor[ibatch, chan_out, window] = qconv

        output_tensor = output_tensor.view(x.shape[0], self.out_channels, 28, 28)
        
        return output_tensor

In [8]:
class Qnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.myconv = QuantumConv(1,8)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28*8,64)
        self.fc2 = nn.Linear(64,10)

    def forward(self,x):

        x = self.myconv(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [9]:
customModel = Qnet()
opt = torch.optim.Adam(customModel.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train(train_dl, customModel, loss_func, opt)
    test(test_dl, customModel, loss_func)